<a href="https://colab.research.google.com/github/AruneshDev/Analyzing-Historical-Stock-Revenue-Data-and-Building-a-Dashboard/blob/main/World_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Fetching a live stock data

In [8]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [12]:
import yfinance as yf
import pandas as pd
import plotly.express as px

# Define top 20 tech stock tickers
top_tech_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "ADBE", "NFLX", "CRM",
    "AMD", "INTC", "ORCL", "IBM", "CSCO", "UBER", "PYPL", "SHOP", "SQ", "SNOW"
]
# Function to get live stock data and market cap
def get_stock_data(tickers):
    data = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1d", interval="5m")  # 5-minute interval
            latest = hist.iloc[-1]  # Get the most recent data

            data.append({
                "Ticker": ticker,
                "Price": latest["Close"],
                "Change (%)": ((latest["Close"] - latest["Open"]) / latest["Open"]) * 100,
                "Market Cap": info.get("marketCap", 0)  # Get market cap (0 if unavailable)
            })
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)
# Fetch stock data
df = get_stock_data(top_tech_tickers)

# Convert Market Cap to Billions
df["Market Cap (B)"] = df["Market Cap"] / 1e9  # Convert to billions

# Display DataFrame
print(df)

Error fetching data for SQ: list index out of range
   Ticker        Price  Change (%)     Market Cap  Market Cap (B)
0    AAPL   245.589996   -0.117947  3688676720640     3688.676721
1    MSFT   408.140015   -0.115017  3034624819200     3034.624819
2   GOOGL   179.660004    0.038982  2200601362432     2200.601362
3    AMZN   216.490005   -0.018470  2295249764352     2295.249764
4    TSLA   337.540009   -0.062174  1086540414976     1086.540415
5    NVDA   134.270004   -0.533374  3292190474240     3292.190474
6    META   683.640015   -0.118343  1731883171840     1731.883172
7    ADBE   444.230011   -0.038250   193412497408      193.412497
8    NFLX  1002.489990   -0.060810   429104431104      429.104431
9     CRM   309.779999    0.093705   296478572544      296.478573
10    AMD   110.790001   -0.261074   179613990912      179.613991
11   INTC    24.850000   -0.699298   107687100416      107.687100
12   ORCL   167.869995   -0.136830   469357854720      469.357855
13    IBM   261.579987  

### Stock Real-Time Heatmap

In [13]:
fig = px.treemap(df,
                 path=["Ticker"],
                 values="Market Cap (B)",  # Market Cap as size
                 color="Change (%)",  # Color based on percentage change
                 color_continuous_scale=["red", "white", "green"],
                 title="Live Tech Stock Heatmap (Sized by Market Cap)")

fig.show()


### Streamlit Dashboard

In [11]:
import streamlit as st
import time

st.title("Live Stock Dashboard")

# Refresh stock data every 10 seconds
while True:
    df = get_stock_data(tickers)
    fig = px.treemap(df, path=["Ticker"], values="Price", color="Change",
                     color_continuous_scale=["red", "white", "green"],
                     title="Live Stock Market Heatmap")

    st.plotly_chart(fig)
    time.sleep(10)  # Refresh every 10 seconds


2025-02-22 05:32:21.903 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:21.997 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-22 05:32:21.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:22.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:22.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:22.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:22.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 05:32:32.700 Thread 'MainThread': mi

KeyboardInterrupt: 

### Investment Calculator

In [ ]:
def calculate_returns(amount, years, expected_return=10):
    return amount * (1 + expected_return / 100) ** years

st.sidebar.header("Investment Calculator")
amount = st.sidebar.number_input("Enter Investment Amount ($)", min_value=100)
years = st.sidebar.slider("Investment Duration (Years)", 1, 10)
returns = calculate_returns(amount, years)

st.sidebar.write(f"Estimated Returns: ${returns:.2f} in {years} years")
